# 步驟一、找出想抓的新聞種類
- 直接從熱門去抓
- 熱門表連結 https://news.ltn.com.tw/list/breakingnews/popular
- 熱門李連杰 https://news.ltn.com.tw/ajax/breakingnews/popular/4


# 步驟二、從網頁原始碼觀察是何種網頁類型
- doodle形式

In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import json

In [61]:
table = {
    "Headline" : [],
    "Link" : [],
    "Date" : [],
    "Source" : [],
    "Category" : [],
}

# 步驟三、先以一頁形式觀察規律

In [62]:
def ltn_scraper(url, page_max):
    # page = 1
    for page in range(1, page_max): 
        url_ = url + str(page)
        print(url_)
        response = requests.get(url_)
        # response.encoding = "utf-8"
        newslists = json.loads(response.text)
        # html = BeautifulSoup(response.text)
        # html
        for num in range(page*20-20, page*20):
            try:
                newslists_nums = newslists["data"][str(num)] 
                headline = newslists_nums["title"]
                table["Headline"].append(headline)
                print("標題:", headline)
            except TypeError:
                newslists_nums = newslists["data"][num] # page1是數字形式
                print("=====TypeError=====")
                headline = newslists_nums["title"]
                table["Headline"].append(headline)
                print("標題:", headline)
            category = newslists_nums["type_en"]
            table["Category"].append(category)
            print("類別:", category)
            date = newslists_nums["time"]
            table["Date"].append(date)
            print("日期:", date)
            link = newslists_nums["url"]
            table["Link"].append(link)
            print("連結:", link)
            source = "自由時報"
            table["Source"].append(source)
            print("來源:", source)
            print("=" * 25)

url = "https://news.ltn.com.tw/ajax/breakingnews/popular/"
ltn_scraper(url, 26)

https://news.ltn.com.tw/ajax/breakingnews/popular/1
=====TypeError=====
標題: 王力宏急了…遭爆找「演藝圈大姐大」施壓！周玉蔻全說了
類別: entertainment
日期: 2021/12/19 19:11
連結: https://ent.ltn.com.tw/news/breakingnews/3773913
來源: 自由時報
=====TypeError=====
標題: 王力宏發聲了！駁斥李靚蕾「一毛都不要」 抖內幕：已經毀了我
類別: entertainment
日期: 2021/12/19 23:22
連結: https://ent.ltn.com.tw/news/breakingnews/3774066
來源: 自由時報
=====TypeError=====
標題: 羽球世錦賽》戴資穎苦戰兩局不敵山口茜 收下銀牌仍平台灣最佳
類別: sports
日期: 2021/12/19 19:04
連結: https://sports.ltn.com.tw/news/breakingnews/3773920
來源: 自由時報
=====TypeError=====
標題: 李靚蕾反擊！爆Yumi換裸照聯絡王力宏
類別: entertainment
日期: 2021/12/19 21:06
連結: https://ent.ltn.com.tw/news/breakingnews/3773989
來源: 自由時報
=====TypeError=====
標題: 李靚蕾深夜再回擊！爆王力宏「遭心理師認定性成癮」
類別: entertainment
日期: 01:42
連結: https://ent.ltn.com.tw/news/breakingnews/3774094
來源: 自由時報
=====TypeError=====
標題: 李靚蕾超硬背景曝光！打臉王家「數億贍養費」說法
類別: entertainment
日期: 2021/12/19 18:03
連結: https://ent.ltn.com.tw/news/breakingnews/3773850
來源: 自由時報
=====TypeError=====
標題: 第18輪疫苗12/16起開打 各地方祭出獎勵措施
類別: life

In [63]:
# page = 1, num = 0-19(range(0, 20))
# page = 2, num = 20-39(range(20, 40))
# page = 3, num = 40-59(range(40, 60))

for page in range(1, 4):
    print("===", page, "===")
    for num in range(page*20-20, page*20):
        print(num)

=== 1 ===
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
=== 2 ===
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
=== 3 ===
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


# 步驟四、套用步驟三的程式到其他頁數，應且將資料存到table

# 步驟五、把table用pandas形式顯示

In [64]:
import pandas as pd

# 解決欄位資料有沒抓成功的資料而無法轉成df顯示的問題
df = pd.DataFrame.from_dict(table, orient = "index")
df = df.transpose() # 將df轉置
df.tail(5) # 取倒數5筆資料

,Headline,Link,Date,Source,Category
495,自由廣場》﹙青年時評﹚公投輸贏？ 評朱立倫談話,https://talk.ltn.com.tw/article/paper/1491063,05:30,自由時報,opinion
496,法說會傳佳音 麗臺開盤半小時內漲停鎖死,https://ec.ltn.com.tw/article/breakingnews/377...,10:20,自由時報,business
497,彭帥現身上海？ 同框姚明,https://news.ltn.com.tw/news/world/paper/1491120,05:30,自由時報,world
498,韓國11月尿素進口量較去年同期減4成,https://ec.ltn.com.tw/article/breakingnews/377...,09:51,自由時報,business
499,高球》杭州亞運高爾夫職業球員參賽有譜？王政松提出看法,https://sports.ltn.com.tw/news/breakingnews/37...,2021/12/19 16:34,自由時報,sports


# 存檔

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
# 存成csv檔
import os
path = "/content/drive/Shareddrives/Media Framing/datasets/ltn/"
if not os.path.exists(path):
    os.makedirs(path)

df.to_csv(path + "ltn_1.csv", encoding = "utf-8", index = False)

# 檢查有無真的存檔

In [67]:
df_ = pd.read_csv("/content/drive/Shareddrives/Media Framing/datasets/ltn/ltn_1.csv")
print(df_.info())
df = pd.read_csv("/content/drive/Shareddrives/Media Framing/datasets/ltn/ltn.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  500 non-null    object
 1   Link      500 non-null    object
 2   Date      500 non-null    object
 3   Source    500 non-null    object
 4   Category  500 non-null    object
dtypes: object(5)
memory usage: 19.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483 entries, 0 to 482
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  480 non-null    object
 1   Link      480 non-null    object
 2   Date      480 non-null    object
 3   Source    480 non-null    object
 4   Category  480 non-null    object
dtypes: object(5)
memory usage: 19.0+ KB
None
